Import Library Yang dibutuhkan

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
from datetime import datetime as dt

from sklearn.preprocessing import RobustScaler,LabelEncoder, MultiLabelBinarizer, OneHotEncoder
from sklearn.metrics import root_mean_squared_error
# from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
from scipy.stats import  shapiro
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split 
from surprise.accuracy import rmse
#from feature_engine.outliers import winsorizers

In [ ]:
df = pd.read_csv("C:/Users/NicholasAH/OneDrive - Bina Nusantara/KULIAHHHHHH/Machine Learning/movies_dataset.csv/movies_dataset.csv")
df.head(10)

,Unnamed: 0,User_Id,Movie_Name,Rating,Genre
0,0,1,Pulp Fiction (1994),5.0,Comedy|Crime|Drama|Thriller
1,1,1,Three Colors: Red (Trois couleurs: Rouge) (1994),3.5,Drama
2,2,1,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,Drama
3,3,1,Underground (1995),5.0,Comedy|Drama|War
4,4,1,Singin' in the Rain (1952),3.5,Comedy|Musical|Romance
5,5,1,Dirty Dancing (1987),4.0,Drama|Musical|Romance
6,6,1,Delicatessen (1991),3.5,Comedy|Drama|Romance
7,7,1,Ran (1985),3.5,Drama|War
8,8,1,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",5.0,Drama
9,9,1,"Bridge on the River Kwai, The (1957)",4.0,Adventure|Drama|War


In [7]:
df = df.drop(columns = ['Unnamed: 0'])

In [8]:
num_cols = []
cat_cols = []

for i in df.columns :
    if 'int' in str(df[i].dtype) or 'float' in str(df[i].dtype):
        num_cols.append(i)
    else :
        cat_cols.append(i)
print(num_cols)
print(cat_cols)

['User_Id', 'Rating']
['Movie_Name', 'Genre']


In [9]:
import re

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

In [10]:
df['Movie_Title'] = df['Movie_Name'].apply(clean_title)

In [11]:
df

,User_Id,Movie_Name,Rating,Genre,Movie_Title
0,1,Pulp Fiction (1994),5.0,Comedy|Crime|Drama|Thriller,Pulp Fiction 1994
1,1,Three Colors: Red (Trois couleurs: Rouge) (1994),3.5,Drama,Three Colors Red Trois couleurs Rouge 1994
2,1,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,Drama,Three Colors Blue Trois couleurs Bleu 1993
3,1,Underground (1995),5.0,Comedy|Drama|War,Underground 1995
4,1,Singin' in the Rain (1952),3.5,Comedy|Musical|Romance,Singin in the Rain 1952
...,...,...,...,...,...
25000090,162541,Ratatouille (2007),4.5,Animation|Children|Drama,Ratatouille 2007
25000091,162541,Bee Movie (2007),2.5,Animation|Comedy,Bee Movie 2007
25000092,162541,Alvin and the Chipmunks (2007),2.0,Children|Comedy,Alvin and the Chipmunks 2007
25000093,162541,"Dark Knight, The (2008)",4.0,Action|Crime|Drama|IMAX,Dark Knight The 2008


In [12]:
df.isnull().sum()

User_Id        0
Movie_Name     0
Rating         0
Genre          0
Movie_Title    0
dtype: int64

In [13]:
df.duplicated().sum()

15

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
cat_cols = []
num_cols = []

for i in df.columns:
    # print(str(df[i].dtype))
    if 'int' in str(df[i].dtype) or 'float' in str(df[i].dtype):
        num_cols.append(i)
    else:
        cat_cols.append(i)

print(f'Categorical: {cat_cols}')
print(f'Numeric: {num_cols}')

Categorical: ['Movie_Name', 'Genre', 'Movie_Title']
Numeric: ['User_Id', 'Rating']


## Label encoding for 'Movie_Name' and 'Genre'

In [16]:
label_encode = LabelEncoder()
mlb = MultiLabelBinarizer()

df['User_Id'] = label_encode.fit_transform(df['User_Id'])
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Genre').str.split('|')), columns= mlb.classes_, index= df.index))

In [17]:
df.drop(columns="(no genres listed)", inplace= True)
df.head()

,User_Id,Movie_Name,Rating,Movie_Title,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Pulp Fiction (1994),5.0,Pulp Fiction 1994,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
1,0,Three Colors: Red (Trois couleurs: Rouge) (1994),3.5,Three Colors Red Trois couleurs Rouge 1994,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,Three Colors Blue Trois couleurs Bleu 1993,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,Underground (1995),5.0,Underground 1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,Singin' in the Rain (1952),3.5,Singin in the Rain 1952,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0


In [18]:
# df.drop(columns="Movie_Name", inplace= True)

## DATA SPLITTING


In [19]:
reader = Reader(rating_scale=(0.5, 5)) 
data = Dataset.load_from_df(df[['User_Id', 'Movie_Title', 'Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


 Kami memakai algoritma SVD karena faktorisasi amtrix dalam sebuah rekomendasi sistem lebih lancar ketika menggunakan SVD. Khususnya dalam prediksi item kepada user berdasarkan rating dan feature lainnya

In [20]:
model = SVD()
model.fit(trainset)

predictions = model.test(testset)


In [21]:
df = df.drop_duplicates(subset=['Movie_Title'])
# unrated_movies = list(set(movie for movie in all_movies if movie not in rated_movies))


Membuat sebuah dataset yang berisikan film2

In [22]:
# Extract all unique movie IDs (or titles) from the dataset
all_movies = df['Movie_Title'].unique().tolist()

Membuat sebuah function yang menghasilkan rekomendasi film kepada user dengan memberikan id user, lalu sistem akan membuat prediksi movie berdasarkan feature2 yang telah diberikan sehingga akan memberikan 5 film yang cocok dengan behaviour user target seccara personal

In [23]:
def get_top_recommendations(user_id, n=5):
    # List of movies the user has already rated
    rated_movies = df[df['User_Id'] == user_id]['Movie_Title'].tolist()

    # Movies the user has not yet rated
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]

    # Create user-movie pairs for prediction
    user_movie_pairs = [(user_id, movie, 0) for movie in unrated_movies]

    # Get predictions for unrated movies
    collaborative_predictions = model.test(user_movie_pairs)

    # Sort predictions by estimated rating in descending order
    top_n_recommendations = sorted(collaborative_predictions, key=lambda x: x.est, reverse=True)[:n]

    # Extract the movie IDs (titles or identifiers) for the top recommendations
    top_n_movie_ids = [pred.iid for pred in top_n_recommendations]
    
    return top_n_movie_ids


Contoh penggunaan sistem

In [24]:
from re import U

# Get recommendations for a user
user_id = 1002
recommendations = get_top_recommendations(user_id)

# Display recommended movie titles
top_n_movies_title = df[df['Movie_Title'].isin(recommendations)]['Movie_Title'].head(5)

for i, title in enumerate(top_n_movies_title, 1):
    print(f"{i}. {title}")


1. Zeitgeist The Movie 2007
2. Band of Brothers 2001
3. Last Lions The 2011
4. Rocks Das Rad 2003
5. Mermaid 1997


Membandingkan Baseline dengan Proposed Model

In [25]:
# Baseline model: Global average rating
global_average = df['Rating'].mean()

# Evaluate baseline predictions
baseline_predictions = []
for _, row in df.iterrows():
    baseline_predictions.append(global_average)

# Compare RMSE of baseline and SVD
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse_baseline = sqrt(mean_squared_error(df['Rating'], baseline_predictions))
print(f"Baseline RMSE: {rmse_baseline}")

svd_rmse = rmse(predictions, verbose=False)
print(f"SVD RMSE: {svd_rmse}")

Baseline RMSE: 1.1271181015019391
SVD RMSE: 0.7776788408661067


Konklusi & Perbandingan

Dapat terlihat bahwa Baseline model memiliki RMSE yang lebih tinggi bila dibandingkan dengan model yang telah kami buat, RMSE merupakan Root Mean Squared Error yang mengukur kesalahan prediksi dalam suatu model. Dapat dilihat bahwa model kami memiliki RMSE sebesar 0.7776 yang menandakan bahwa model kami lebih akurat dalam memprediksi film bagi user bila dibandingkan dengan model yang dimiliki oleh baseline.

Tantangan : Dataset ini memiliki feature yang sedikit untuk memprediksi masing2 behaviour dari user2, namun jumlah dataset yang besar menjadi sebuah keuntungan karena model dapat belajar dari berbagai tipe judul dan rating yang telah ditonton oleh masing-masing user sehingga ia dapat memprediksi movie yang disukai oleh user tertentu.

Future Works : Berbagai perusahaan terutama industri per-filman pasti akan menggunakan model tip2 seperti ini untuk memprediksi, mengevaluasi, dan melakukan market research. 
    Bagi perusahaan pengedar fil seperti Netflix,disney,etc, mereka akan menggunakan ini sebagai alat untuk merekomendasikan film kepada user2 mereka sehingga pengalaman menonton menjadi lebih baik.  
    Bagi para produser dan filmmaker, model seperti ini dapat berguna untuk memutuskan genre apa yang dapat menghasilkan revenue dan menggapai pasar yang lebih besar.